In [3]:
import pandas as pd
import pathlib
import yaml
from functions import *
from collections import Counter

import plotly.graph_objects as go

# Path
BASE_PATH = pathlib.Path().resolve()
DATA_PATH = BASE_PATH.joinpath('data').resolve()

# Read Data
df = pd.read_csv(DATA_PATH.joinpath('df.csv'), delimiter=';', skiprows=4, na_values='#')

df = df.to_dict('records')

In [6]:
pd.DataFrame.from_dict(df, orient='columns')

,Work center,Notification,Notif.type,Notif.type.1,Notif.date,Status (Notif.),Service order,Order type,Work center.1,Material (global),...,HIST time,Req.end date,Req.end time,Outstat. Ind.,ConTAT,ConTAT ac.,Equ TAT,Equ TAT ac.,Shp TAT,ShTAT Ac.
0,2000/CELL 6,302405037,KT,Disposition Notif,29.10.2019,NOPR ORAS OSTS,83432074,ZR06,Cell 6 Hydro-Mechanics,971751:62983,...,NaN,NaN,NaN,Not assigned,3,0,0,0,0,0
1,2000/EB,302330576,K1,Component Induction,31.08.2019,NOCO NOPT ATCO SOAS,83413302,ZR05,EB INDICATORS 2,ASP7330:04YP3,...,15:06:06,16.09.2019,14:15:07,Not assigned,17,13,2,1,14,9
2,2000/EB,302330759,K1,Component Induction,01.09.2019,NOCO NOPT ATCO SOAS,83413291,ZR05,EB INDICATORS 2,4305990050:F0217,...,13:40:09,17.09.2019,14:03:31,Not assigned,17,12,1,1,14,9
3,2000/EB,302332856,K1,Component Induction,03.09.2019,NOCO NOPT ATCO SOAS,83415870,ZR05,EB INDICATORS 2,WL102AMS3:K5294,...,13:11:21,25.09.2019,09:45:34,Y,23,0,2,0,14,0
4,2000/EB,302333737,K1,Component Induction,03.09.2019,NOCO NOPT ATCO SOAS,83413728,ZR05,EB INDICATORS 2,APTE8A20007BARD:58828,...,07:26:28,19.09.2019,13:36:35,Not assigned,17,16,2,0,14,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49090,2000/WP,302554913,K1,Component Induction,09.03.2020,NOPR NOPT OSTS SOAS,83471041,ZR05,shop Parijs,3215618-5:59364,...,NaN,24.03.2020,NaN,Y,25,0,16,0,16,0
49091,2000/WP,302556177,KT,Disposition Notif,31.01.2020,ORAS NOCO ATCO,83471037,ZR06,shop Parijs,63292146-1:59364,...,20:53:32,20.02.2020,NaN,Not assigned,27,3,8,1,24,1
49092,2000/WP,302561031,KT,Disposition Notif,05.02.2020,ORAS NOCO ATCO,83472402,ZR06,shop Parijs,63292146-1:59364,...,13:46:10,01.03.2020,NaN,Y,27,3,8,1,24,1
49093,2000/WP,302564778,K1,Component Induction,18.03.2020,NOPR NOPT OSTS SOAS,83473513,ZR05,shop Parijs,3215618-5:59364,...,NaN,05.04.2020,NaN,Not assigned,19,0,16,0,16,0
